# Vault Transfers

*Work in Progress*

This notebook aims to provide a brief demonstration of how this codebase could be used for visualize vault transfers.

In [1]:
"""load dependencies and define constants"""
import os
import sys
import numpy as np
import pandas as pd
from decimal import Decimal
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
from typing import Literal

sys.path.append("..")
from src.yearn import Network, Yearn, Subgraph, Vault
from src.utils.web3 import Web3Provider 
from src.utils.network import client, parse_json

# constants
ZERO_ADDRESS = '0x0000000000000000000000000000000000000000'
FROM_BLOCK = 13900000  # Dec-29-2021
BATCH_SIZE = 500000
API_ENDPOINT = "https://api.etherscan.io/api"

pd.set_option("expand_frame_repr", False)

In [2]:
"""get current block and load vaults from the yearn instance"""
load_dotenv()
w3 = Web3Provider(Network.Mainnet)
current_block = w3.provider.eth.get_block("latest")["number"]
print(f"current block number: {current_block}")

yearn = Yearn(Network.Mainnet)
vaults = yearn.vaults
print(f"loaded metadata for {len(vaults)} vaults (v2)")

subgraph = Subgraph(Network.Mainnet)

current block number: 14950522
loaded metadata for 101 vaults (v2)


In [3]:
"""get addresses that withdrew from vaults"""
withdrawals_stats = {"vaults": {}, "overall_account_transfers": {} }
stat_vals = ["count", "shares", "shares_usdc"]
for idx, vault in enumerate(vaults):
    # approximate the share price by the current usdc value, skip if not available
    try:
        share_price = w3.get_usdc_price(vault.token.address)
    except:
        continue

    transfers = subgraph.transfers("withdrawals", vault, from_block=FROM_BLOCK, to_block=current_block)

    # vault's count and shares stats
    _dict = {
        "count": transfers.count,
        "shares": float(transfers.shares),
        "shares_usdc": float(transfers.shares * share_price),
        "account_transfers": transfers.account_transfers
    }
    print(f"vault: {vault.address} {dict((k, v) for k, v in _dict.items() if k in stat_vals)}")

    # add vault to withdrawals stats
    withdrawals_stats["vaults"][vault.address] = _dict

    account_items = transfers.account_transfers.items()
    account_transfers = sorted(account_items, key=lambda item: item[1].shares, reverse=True)
    
    for index, (account, transfer) in enumerate(account_transfers[:10]):
        # vault's account stats
        _dict = {
            "count": transfer.count,
            "shares": float(transfer.shares),
            "shares_usdc": float(transfer.shares * share_price)
        }
        print(f"{index}: {account} {_dict}")

        # add account_transfers to withdrawals stats
        overall_account_transfers = withdrawals_stats["overall_account_transfers"]
        if overall_account_transfers.get(account) is None:
            overall_account_transfers[account] = {}
        account_stats = overall_account_transfers[account]
        overall_account_transfers[account] = {k: account_stats.get(k, 0) + _dict[k] for k in _dict}
    print()

# withdrawals stats for vaults and account_transfers
for key in withdrawals_stats.keys():
    vault_stats = []
    for subkey, subval in withdrawals_stats[key].items():
        # only include items in stat_vals, plus address
        new_val = dict((k, v) for k, v in subval.items() if k in stat_vals)
        vault_stats.append((subkey, *new_val.values()))
    df = pd.DataFrame(vault_stats, columns=["address", "count", "shares", "approx. USDC value"])
    df["approx. USDC value"] = pd.to_numeric(df["approx. USDC value"])
    df.set_index(keys=["address"], inplace=True)
    df.sort_values("approx. USDC value", inplace=True, ascending=False)
    print(f"top {key}")
    print(df.head(10))
    print()

vault: 0x5f18C75AbDAe578b483E5F43f12a39cF75b973a9 {'count': 939, 'shares': 113303176.195184, 'shares_usdc': 113042805.49628747}
0: 0x99fd1378ca799ed6772fe7bcdc9b30b389518962 {'count': 4, 'shares': 29945931.328055, 'shares_usdc': 29877115.57786313}
1: 0xd1370566b1d0ffc9cfbb5f365de4f9f63c134afd {'count': 6, 'shares': 25251349.463725, 'shares_usdc': 25193321.86265736}
2: 0x1c0b104a9eeff2f7001348a49fa28b8a0d23d637 {'count': 1, 'shares': 10525590.682641, 'shares_usdc': 10501402.875252292}
3: 0x1824df8d751704fa10fa371d62a37f9b8772ab90 {'count': 103, 'shares': 9548288.538278, 'shares_usdc': 9526346.571217038}
4: 0xfeb4acf3df3cdea7399794d0869ef76a6efaff52 {'count': 1, 'shares': 4888205.419409, 'shares_usdc': 4876972.323355198}
5: 0xca78478456b044ac13954a9172583dd1737e52d2 {'count': 52, 'shares': 4593501.906628, 'shares_usdc': 4582946.039246568}
6: 0xbabee6d5f6edd301b5fae591a0d61ab702b359d0 {'count': 176, 'shares': 1955724.110131, 'shares_usdc': 1951229.856125919}
7: 0x782bc9b1f11cdba13acb030cd

2022-06-12 22:01:27.928 WARNING connectionpool: Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='api.etherscan.io', port=443): Read timed out. (read timeout=10)")': /api?&apiKey=5RW3TIB9G6X4Y9NP136F1PHJNFP76XNH63&address=0x83d95e0D5f402511dB06817Aff3f9eA88224B030&module=contract&action=getabi


vault: 0x7047F90229a057C13BF847C0744D646CFb6c9E1A {'count': 8, 'shares': 25.387834213888432, 'shares_usdc': 705512.8696997771}
0: 0x386d15ee0df3fa0f1b909b8bfa2493cbc5161b1d {'count': 1, 'shares': 20.545312581269624, 'shares_usdc': 570942.0628783284}
1: 0xc9e7a40b20e2b10b3619109a74e9e6ef17c31299 {'count': 1, 'shares': 1.938149055895052, 'shares_usdc': 53860.01384798711}
2: 0x1fe14c72bd05a61dc8ee3fdda6633dc5a1b6bbe6 {'count': 1, 'shares': 0.9727360467402667, 'shares_usdc': 27031.758361676766}
3: 0x6a999aa95842c89564c19f721cc96c67a7d7b8c6 {'count': 1, 'shares': 0.9629847583584205, 'shares_usdc': 26760.775835495664}
4: 0xd291328a6c202c5b18dcb24f279f69de1e065f70 {'count': 2, 'shares': 0.49195378786070976, 'shares_usdc': 13671.104266287297}
5: 0xbb3729c979ec6434ab30709a34d31134f1e3db0b {'count': 1, 'shares': 0.43502589389608515, 'shares_usdc': 12089.111824609243}
6: 0xd94dcc3d3ce6e173866939f09d3d666e75cc0070 {'count': 1, 'shares': 0.04167208986827314, 'shares_usdc': 1158.042685392557}

vault

In [4]:
"""get addresses that deposit to vaults"""
deposits_stats = { "vaults": {}, "overall_account_transfers": {} }
stat_vals = ["count", "shares", "shares_usdc"]
for idx, vault in enumerate(vaults):
    # approximate the share price by the current usdc value, skip if not available
    try:
        share_price = w3.get_usdc_price(vault.token.address)
    except:
        continue
    
    transfers = subgraph.transfers("deposits", vault, from_block=FROM_BLOCK, to_block=current_block)

    # vault's count and shares stats
    _dict = {
        "count": transfers.count,
        "shares": float(transfers.shares),
        "shares_usdc": float(transfers.shares * share_price),
        "account_transfers": transfers.account_transfers
    }
    print(f"vault: {vault.address} {dict((k, v) for k, v in _dict.items() if k in stat_vals)}")

    # add vault to deposits stats
    deposits_stats["vaults"][vault.address] = _dict

    account_items = transfers.account_transfers.items()
    account_transfers = sorted(account_items, key=lambda item: item[1].shares, reverse=True)
    
    for index, (account, transfer) in enumerate(account_transfers[:10]):
        # vault's account stats
        _dict = {
            "count": transfer.count,
            "shares": float(transfer.shares),
            "shares_usdc": float(transfer.shares * share_price)
        }
        print(f"{index}: {account} {_dict}")

        # add account_transfers to deposits stats
        overall_account_transfers = deposits_stats["overall_account_transfers"]
        if overall_account_transfers.get(account) is None:
            overall_account_transfers[account] = {}
        account_stats = overall_account_transfers[account]
        overall_account_transfers[account] = {k: account_stats.get(k, 0) + _dict[k] for k in _dict}
    print()

# deposits stats for vaults and account_transfers
for key in deposits_stats.keys():
    vault_stats = []
    for subkey, subval in deposits_stats[key].items():
        # only include items in stat_vals, plus address
        new_val = dict((k, v) for k, v in subval.items() if k in stat_vals)
        vault_stats.append((subkey, *new_val.values()))
    df = pd.DataFrame(vault_stats, columns=["address", "count", "shares", "approx. USDC value"])
    df["approx. USDC value"] = pd.to_numeric(df["approx. USDC value"])
    df.set_index(keys=["address"], inplace=True)
    df.sort_values("approx. USDC value", inplace=True, ascending=False)
    print(f"top {key}")
    print(df.head(10))
    print()

vault: 0x5f18C75AbDAe578b483E5F43f12a39cF75b973a9 {'count': 136, 'shares': 9037357.051826, 'shares_usdc': 9016589.205320904}
0: 0xd1370566b1d0ffc9cfbb5f365de4f9f63c134afd {'count': 1, 'shares': 5709246.196335, 'shares_usdc': 5696126.348575822}
1: 0xdea04ffc66ecd7bf35782c70255852b34102c3b0 {'count': 92, 'shares': 2583416.319372, 'shares_usdc': 2577479.6286700834}
2: 0x97990b693835da58a281636296d2bf02787dea17 {'count': 1, 'shares': 684270.578476, 'shares_usdc': 682698.1246866621}
3: 0xee91ae665be6a92a1241911cea3cae9d50fe658d {'count': 20, 'shares': 40956.823981, 'shares_usdc': 40862.70519949166}
4: 0x5c8507d4aa2a0ebe7fa79fcf689fbb07f8007700 {'count': 1, 'shares': 9133.809901, 'shares_usdc': 9112.820405847502}
5: 0x92be6adb6a12da0ca607f9d87db2f9978cd6ec3e {'count': 1, 'shares': 4416.697153, 'shares_usdc': 4406.547582942406}
6: 0x903fd974073761143a028e2f8341a93dd063d2a4 {'count': 1, 'shares': 1097.899634, 'shares_usdc': 1095.376660641068}
7: 0x155357505afa570e6d8e2c3eebbdebd1dd1f0bf7 {'cou

In [5]:
net_vaults = {}
for i, j in zip(withdrawals_stats["vaults"], deposits_stats["vaults"]):
    pass